In [3]:
import pandas as pd
import numpy as np
import stringcompare

In [56]:
#read
result = pd.read_csv('data/patents_2005_012_autosequence.csv', index_col = 0, dtype={'patent': 'string', 'fname': 'string', 'mname': 'string', \
    'lname': 'string', 'suffix': 'string'})

#clean (convert sequence to int, omit NaN's)
result['sequence'] = pd.to_numeric(result['sequence'], errors='ignore').astype(pd.Int16Dtype())
result = result[result['sequence'] != "NaN"]

#get mention id
result['mention'] = result.apply(lambda x: "US" + str(x.patent) + "-" + str(x.sequence), axis=1)
#result.set_index('mention', inplace=True)

#find duplicates and create slice for re-assigning sequence number manually
todo = result[result.mention.duplicated(keep=False)].copy()
finished = result[result.mention.duplicated(keep=False) == False].copy()

In [ ]:
#reading in raw_inventor (and sorting for efficiency)
rawinventor = pd.read_csv("data/rawinventor.tsv", sep="\t", usecols=["patent_id", "sequence", "name_first", "name_last"], dtype={"patent_id": "string", "sequence": "int16", "name_first": "string", "name_last": "string"})
rawinventor.set_index(['patent_id', 'sequence'], inplace=True)
rawinventor.sort_index(inplace=True)
rawinventor

In [57]:
#for fixing manually pt. 1
todo.sort_values('patent', inplace=True)
todo.reset_index(inplace=True)
counter = 0
patent = todo.patent.loc[counter]
rawinventor.loc[patent]

,name_first,name_last
sequence,,
0,C. -H.,Chang
1,Claude F.,Meares
2,C.-H,Chang


In [79]:
#for fixing manually pt. 2
counter += 2
patent = todo.patent.loc[counter]
print(patent)
rawinventor.loc[patent]

6830895


,name_first,name_last
sequence,,
0,Gordon A.,Andrews
1,Joseph E.,Smith


In [33]:
#writing to csv for manual inspection
todo.to_csv('todo.csv')

In [91]:
#reading cleaned dataframe
done = pd.read_csv('todo.csv', index_col = 0)

#clean (convert sequence to int, omit NaN's)
done['sequence'] = pd.to_numeric(done['sequence'], errors='ignore').astype(pd.Int16Dtype())
done = done[done['sequence'] != "NaN"]

#drop existing duplicates (beyond our control)
done.drop_duplicates(inplace=True)

#writing mention
done['mention'] = done.apply(lambda x: "US" + str(x.patent) + "-" + str(x.sequence), axis=1)

In [101]:
#concatenating with already correct (non duplicated) rows and saving dataframe
final = pd.concat([finished, done])
final.set_index('mention')
final.to_csv('data/autosequence_cleaned.csv')